In [3]:
from google.colab import files
uploaded = files.upload()

Saving energydata_complete.csv to energydata_complete.csv


In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
path = '/content/energydata_complete.csv'
energy_data = pd.read_csv(path)
energy_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [14]:
energy_data.isnull()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19731,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19732,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19733,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [28]:
#removing extra column which are not useful
energy_data = energy_data.drop(['date', 'lights'], axis = 1)

In [32]:
scaler = MinMaxScaler()
normalized_energy_data = pd.DataFrame(scaler.fit_transform(energy_data), columns = energy_data.columns)
normalized_energy_data.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.046729,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,0.381691,0.381070,0.841827,0.170594,0.653428,0.173329,0.661412,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.046729,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,0.381691,0.375443,0.839872,0.170594,0.651064,0.173329,0.660155,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.037383,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,0.380037,0.367487,0.830704,0.170594,0.646572,0.173329,0.655586,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.037383,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,0.380037,0.363800,0.833401,0.164310,0.641489,0.164175,0.650788,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.046729,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,0.380037,0.361859,0.848264,0.170594,0.639362,0.164175,0.650788,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


In [33]:
X = normalized_energy_data.drop(['Appliances'], axis = 1)
y = normalized_energy_data["Appliances"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)


In [34]:
x1 = np.array(normalized_energy_data['T2'])
y1 = np.array(normalized_energy_data['T6'])
x1 = x1.reshape(-1, 1)
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size = 0.3, random_state = 42)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x1_train, y1_train)
y1_pred = model.predict(x1_test)

from sklearn.metrics import r2_score
r2 = r2_score(y1_test, y1_pred)
round(r2, 2)

0.64

In [41]:
reg = LinearRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
train_score = reg.score(X_train, y_train)
test_score = reg.score(X_test, y_test)
print(f'linear regression model training score is {train_score}')
print(f'linear regression model testing score is {test_score}')


linear regression model training score is 0.14471942308518737
linear regression model testing score is 0.14890246319303535


In [44]:
from sklearn.metrics import mean_absolute_error
m_absolute_error = mean_absolute_error(y_test, y_pred)
print("mean absolute error {}".format(round(m_absolute_error,2)))

mean absolute error 0.05


In [45]:
RSS = np.sum(np.square(y_test - y_pred))
print('Residual Sum of Squares for linear model: {}'.format(round(RSS, 2)))

Residual Sum of Squares for linear model: 45.35


In [48]:
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, y_pred)
root_MSE = np.sqrt(MSE)
print('Root Mean Squared Error for linear model: {}'.format(round(root_MSE, 3)))

Root Mean Squared Error for linear model: 0.088


In [49]:
from sklearn.metrics import r2_score
rsquared = r2_score(y_test, y_pred)
print('R_Squared for linear model: {}'.format(round(rsquared, 2)))

R_Squared for linear model: 0.15


In [50]:
def weights(model, feat, col_name):
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_reg_weights = weights(reg, X_train, 'Linear_model_Weight')
linear_reg_weights

,Features,Linear_model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [51]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 0.4)
ridge.fit(X_train, y_train)
ridge_pred = ridge.predict(X_test)
train_score = ridge.score(X_train, y_train)
test_score = ridge.score(X_test, y_test)
print("Training score of ridge regression model is {:.3f}".format(train_score))
print("Testing score of ridge regression model is {:.3f}".format(test_score))

Training score of ridge regression model is 0.145
Testing score of ridge regression model is 0.149


In [54]:
from sklearn.linear_model import Lasso
lasso0001 = Lasso(alpha = 0.001)
lasso0001.fit(X_train, y_train)
lasso0001_pred = lasso0001.predict(X_test)
train_score = lasso0001.score(X_train, y_train)
test_score = lasso0001.score(X_test, y_test)
print("lasso regression model for training score  is {:.3f}".format(train_score))
print("regression model for test_Score is {:.3f}".format(test_score))
print("training feautres total amount: {:.3f}".format(np.sum(lasso0001.coef_ != 0)))

lasso regression model for training score  is 0.025
regression model for test_Score is 0.027
training feautres total amount: 4.000


In [58]:
from sklearn.metrics import mean_squared_error
m_square_error = mean_squared_error(y_test, lasso0001_pred)
root_mse = np.sqrt(m_square_error)
print('ridge model root mean square error: {}'.format(round(root_mse, 3)))

ridge model root mean square error: 0.094
